In [5]:
import pandas as pd
df = pd.read_csv('non_severity_cves_20230330.csv')

In [6]:
df.head()

,CVE,num_of_author,baseSeverity,baseScore
0,CVE-2023-25610,278,NaN,NaN
1,CVE-2023-0179,216,NaN,NaN
2,CVE-2023-0210,109,NaN,NaN
3,CVE-2023-24749,96,NaN,NaN
4,CVE-2023-28760,76,NaN,NaN


# -*- coding:utf-8 -*-
from selenium import webdriver

# Chromeで操作する場合
driver = webdriver.Chrome()
driver = webdriver.Chrome(executable_path='chromedriver')
# driver.get('https://www.google.co.jp/')

In [7]:
url = 'https://services.nvd.nist.gov/rest/json/cves/2.0\?cveId\='


In [35]:
import requests
import json
def get_cve_resultsPerPage(cve_id):
    # Initialize
    resultsPerPage = None

    # 入力したCVE番号に対応する脆弱性情報をNVDのAPIからJSON形式で取得して、resultsPerPageを返却する
    api = "https://services.nvd.nist.gov/rest/json/cves/2.0\?cveId="
    cve = cve_id
    uri = api + cve
    # print(uri)
    try:
        response = requests.get(uri)
    except Exception as e :
        raise e
    # print(response.text)
    json_data = json.loads(response.text)
    
    if json_data['resultsPerPage'] == 0 :
        resultsPerPage = json_data['resultsPerPage']
        baseSeverity = None
        baseScore = None
        # print(f"{cve}:resultsPerPage={resultsPerPage}")
    
    elif json_data['resultsPerPage'] == 1 :
        resultsPerPage = json_data['resultsPerPage']
        # print(f"{cve}:resultsPerPage={resultsPerPage}")
        
        # if json_data['vulnerabilities']:
        if json_data['vulnerabilities'][0]['cve']['metrics']:
            try :
                baseSeverity = json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseSeverity']
                baseScore = json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV31'][0]['cvssData']['baseScore']

            except Exception as e :
                print(e)
                baseSeverity = "V2_".join(json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV2'][1]['baseSeverity'])
                baseScore = "V2_".join(json_data['vulnerabilities'][0]['cve']['metrics']['cvssMetricV2'][0]['cvssData']['baseScore'])

        else :
            resultsPerPage = json_data['resultsPerPage']
            baseSeverity = None
            baseScore = None
    
    else :
        resultsPerPage = json_data['resultsPerPage']
        # print(f"{cve}:resultsPerPage={resultsPerPage}")
        baseSeverity = None
        baseScore = None


    return [cve,resultsPerPage,baseSeverity,baseScore]


In [9]:
print(get_cve_resultsPerPage('CVE-2021-25610'))

(0, None, None)


In [10]:
print(get_cve_resultsPerPage('CVE-2021-27365'))

(1, 'HIGH', 7.8)


In [11]:
nonSeverityCVEs_list = df['CVE'].to_list()

In [12]:
print(nonSeverityCVEs_list)

['CVE-2023-25610', 'CVE-2023-0179', 'CVE-2023-0210', 'CVE-2023-24749', 'CVE-2023-28760', 'CVE-2023-2561', 'CVE-2023-28303', 'CVE-2023-20860', 'CVE-2017-9140', 'CVE-2022-22655', 'CVE-2022-36413', 'CVE-2022-23529', 'CVE-2023-0045', 'CVE-2023-24486', 'CVE-2016-10190', 'CVE-2014-12509', 'CVE-2018-5093', 'CVE-2023-0291', 'CVE-2014-12508', 'CVE-2015-2291', 'CVE-2023-26463', 'CVE-2017-0145', 'CVE-2023-2575', 'CVE-2023-0494', 'CVE-2023-1175', 'CVE-2015-10097', 'CVE-2021-33237', 'CVE-2020-19786', 'CVE-2016-15030', 'CVE-2022-43634', 'CVE-2023-1616', 'CVE-2023-0056', 'CVE-2023-27326', 'CVE-2023-27537', 'CVE-2023-25828', 'CVE-2022-38217', 'CVE-2014-8361', 'CVE-2023-1247', 'CVE-2022-42948', 'CVE-2021-3099', 'CVE-2021-3091', 'CVE-2023-28426', 'CVE-2023-28686', 'CVE-2017-8291', 'CVE-2022-37950', 'CVE-2022-37949', 'CVE-2022-37948', 'CVE-2021-29410', 'CVE-2022-37951', 'CVE-2018-3623', 'CVE-2021-23185', 'CVE-2021-23199', 'CVE-2018-17144', 'CVE-2023-1289', 'CVE-2018-12136', 'CVE-2018-12134', 'CVE-2018-12

In [13]:
len(nonSeverityCVEs_list)

72

In [37]:
from time import sleep
interval = 6
result = []
for cve in nonSeverityCVEs_list[:]:
    result = get_cve_resultsPerPage(cve)
    sleep(interval)
    print(result)

['CVE-2023-25610', 0, None, None]
['CVE-2023-0179', 1, None, None]
['CVE-2023-0210', 1, None, None]
['CVE-2023-24749', 0, None, None]
['CVE-2023-28760', 0, None, None]
['CVE-2023-2561', 0, None, None]
['CVE-2023-28303', 0, None, None]
['CVE-2023-20860', 1, None, None]
'cvssMetricV31'


IndexError: list index out of range

In [41]:
# https://qiita.com/usop4/items/70b7db1acbd54112c428
import urllib.request
from bs4 import BeautifulSoup
from time import sleep
import logging

# https://www.true-fly.com/entry/2022/03/02/073000
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

for cve in nonSeverityCVEs_list[:3]:
    # リトライ間隔（秒）
    retry_interval = 6
    # 試行回数
    tries = 5
    
    for i in range(0, tries):
        try:
            instance = urllib.request.urlopen("https://nvd.nist.gov/vuln/detail/"+cve)
            break
            
        except Exception as e:
            if i + 1 == tries:
                raise e
            
            sleep(retry_interval)
            logger.info('リトライ回数:{}回目'.format(i + 1))
            continue
            
    soup = BeautifulSoup(instance, "html.parser")
    score = soup.select_one("[data-testid=vuln-cvss3-panel-score]")
    desc = soup.select_one("[data-testid=vuln-cvss3-nist-vector-na]")
    if hasattr(desc, "text"):
        print(f"{cve}, {desc.text.strip()}")
    elif hasattr(score, "text"):
        print(f"{cve}, {score.text.strip()}")
        
    else:
        for i in range(0, tries):
            try:
                instance2 = urllib.request.urlopen("https://cveawg.mitre.org/api/cve-id/"+cve)
                break

            except Exception as e:
                if i + 1 == tries:
                    raise e

                sleep(retry_interval)
                logger.info('リトライ回数:{}回目'.format(i + 1))
                continue
            
        soup2 = BeautifulSoup(instance2, "html.parser")
        desc2 = soup2.select_one("#GeneratedTable > table > tbody > tr:nth-child(4) > td > a")
        if hasattr(desc2, "text"):
            print(f"{cve}, {desc2.text.strip()}")
        

CVE-2023-0179, NVD score
														not yet provided.
CVE-2023-0210, NVD score
														not yet provided.


In [75]:
import requests
import json
def get_cve_reserved(cve_id):
    # Initialize
    resultsPerPage = None

    # 入力したCVE番号に対応する脆弱性情報をNVDのAPIからJSON形式で取得して、resultsPerPageを返却する
    api = "https://cveawg.mitre.org/api/cve-id/"
    cve = cve_id
    uri = api + cve
    # print(uri)
    try:
        response = requests.get(uri)
    except Exception as e :
        raise e
    # print(response.text)
    json_data = json.loads(response.text)
    
    try:
        if json_data['state']:
            return [cve, json_data['state']]
    
    except :
        return [cve, json_data['error']]

In [47]:
print(get_cve_reserved("CVE-2023-25610"))

['CVE-2023-25610', 'RESERVED']


In [69]:
import urllib.request
from bs4 import BeautifulSoup
from time import sleep
import logging

# https://www.true-fly.com/entry/2022/03/02/073000
# 処理時間を計測するため、loggerのフォーマット設定しています
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = '%(asctime)s [%(levelname)s] %(filename)s, lines %(lineno)d. %(message)s'
formatter = logging.Formatter(format, '%Y-%m-%d %H:%M:%S')
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

def get_cve_description(cve_id):
    # https://qiita.com/usop4/items/70b7db1acbd54112c428

    # リトライ間隔（秒）
    retry_interval = 6
    # 試行回数
    tries = 5
    
    for i in range(0, tries):
        try:
            instance = urllib.request.urlopen("https://nvd.nist.gov/vuln/detail/"+cve)
            break

        except Exception as e:
            if i + 1 == tries:
                raise e
            
            sleep(retry_interval)
            logger.info('リトライ回数:{}回目'.format(i + 1))
            continue
            
    soup = BeautifulSoup(instance, "html.parser")
    score = soup.select_one("[data-testid=vuln-cvss3-panel-score]")
    desc = soup.select_one("[data-testid=vuln-cvss3-nist-vector-na]")

    if hasattr(score, "text"):
        return [cve, score.text.strip()]
        # print(f"{cve}, {score.text.strip()}")
    
    elif hasattr(desc, "text"):
        return [cve, desc.text.replace('\r','').replace('\n',' ').replace('\t','')]
        # print(f"{cve}, {desc.text.strip()}")


In [66]:
print(get_cve_description("CVE-2023-0179"))

['CVE-2023-0210', 'NVD score not yet provided.']


In [60]:
print(get_cve_description("CVE-2023-0179"))

['CVE-2023-0210', 'NVD score\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\tnot yet provided.']


In [78]:
from time import sleep
interval = 2
result = []
for cve in nonSeverityCVEs_list[:]:

    desc = get_cve_description(cve)

    
    if desc :
        result.append(desc)
    
    else :
        reserved = get_cve_reserved(cve)   
        sleep(interval)

        
        if reserved :
            result.append(reserved)
    print(result)

[['CVE-2023-25610', 'RESERVED']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.'], ['CVE-2023-0210', 'NVD score not yet provided.']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.'], ['CVE-2023-0210', 'NVD score not yet provided.'], ['CVE-2023-24749', 'RESERVED']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.'], ['CVE-2023-0210', 'NVD score not yet provided.'], ['CVE-2023-24749', 'RESERVED'], ['CVE-2023-28760', 'RESERVED']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.'], ['CVE-2023-0210', 'NVD score not yet provided.'], ['CVE-2023-24749', 'RESERVED'], ['CVE-2023-28760', 'RESERVED'], ['CVE-2023-2561', 'CVE_ID_NOT_FOUND']]
[['CVE-2023-25610', 'RESERVED'], ['CVE-2023-0179', 'NVD score not yet provided.'], ['CVE-2023-0210', 'NVD score not yet provided.'], ['CVE-2023-24749', '

In [79]:
df.tail()

,CVE,num_of_author,baseSeverity,baseScore
67,CVE-2017-0987,10,NaN,NaN
68,CVE-2017-0988,10,NaN,NaN
69,CVE-2017-0989,10,NaN,NaN
70,CVE-2023-28336,10,NaN,NaN
71,CVE-2023-39952,10,NaN,NaN


In [85]:
df_checked = pd.DataFrame(result, columns = ['CVE_checked', 'reason'])
df_checked.tail()

,CVE_checked,reason
67,CVE-2017-0987,NVD score not yet provided.
68,CVE-2017-0988,NVD score not yet provided.
69,CVE-2017-0989,NVD score not yet provided.
70,CVE-2023-28336,NVD score not yet provided.
71,CVE-2023-39952,CVE_ID_NOT_FOUND


In [86]:
df_concat = pd.concat([df, df_checked],axis = 1)
df_concat.head()

,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
0,CVE-2023-25610,278,NaN,NaN,CVE-2023-25610,RESERVED
1,CVE-2023-0179,216,NaN,NaN,CVE-2023-0179,NVD score not yet provided.
2,CVE-2023-0210,109,NaN,NaN,CVE-2023-0210,NVD score not yet provided.
3,CVE-2023-24749,96,NaN,NaN,CVE-2023-24749,RESERVED
4,CVE-2023-28760,76,NaN,NaN,CVE-2023-28760,RESERVED


In [88]:
df_concat.to_csv("checked_non_severity_cves_20230330.csv", header = True)

In [90]:
df2 = pd.read_csv("modified_non_severity_cves_20230330.csv")

In [91]:
df2.head()

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
0,0,CVE-2023-25610,278,NaN,NaN,CVE-2023-25610,RESERVED
1,1,CVE-2023-0179,216,NaN,NaN,CVE-2023-0179,NVD score not yet provided.
2,2,CVE-2023-0210,109,NaN,NaN,CVE-2023-0210,NVD score not yet provided.
3,3,CVE-2023-24749,96,NaN,NaN,CVE-2023-24749,RESERVED
4,4,CVE-2023-28760,76,NaN,NaN,CVE-2023-28760,RESERVED


In [111]:
df2["baseSeverity"].

TypeError: You have to supply one of 'by' and 'level'

In [124]:
df2.query("baseSeverity == 'CRITICAL'")

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
10,10,CVE-2022-36413,31,CRITICAL,9.1,CVE-2022-36413,9.1 CRITICAL
14,14,CVE-2016-10190,26,CRITICAL,9.8,CVE-2016-10190,9.8 CRITICAL
25,25,CVE-2015-10097,16,CRITICAL,9.8,CVE-2015-10097,9.8 CRITICAL
38,38,CVE-2022-42948,12,CRITICAL,9.8,CVE-2022-42948,9.8 CRITICAL


In [129]:
df2[df2["baseSeverity"] == 'CRITICAL']

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
10,10,CVE-2022-36413,31,CRITICAL,9.1,CVE-2022-36413,9.1 CRITICAL
14,14,CVE-2016-10190,26,CRITICAL,9.8,CVE-2016-10190,9.8 CRITICAL
25,25,CVE-2015-10097,16,CRITICAL,9.8,CVE-2015-10097,9.8 CRITICAL
38,38,CVE-2022-42948,12,CRITICAL,9.8,CVE-2022-42948,9.8 CRITICAL


In [130]:
df2[df2["baseSeverity"] == 'MEDIUM']

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
8,8,CVE-2017-9140,47,MEDIUM,6.1,CVE-2017-9140,6.1 MEDIUM
24,24,CVE-2023-1175,17,MEDIUM,6.6,CVE-2023-1175,6.6 MEDIUM
28,28,CVE-2016-15030,15,MEDIUM,6.1,CVE-2016-15030,6.1 MEDIUM
30,30,CVE-2023-1616,14,MEDIUM,5.4,CVE-2023-1616,5.4 MEDIUM
53,53,CVE-2023-1289,10,MEDIUM,5.5,CVE-2023-1289,5.5 MEDIUM


In [125]:
df2.query("baseSeverity == 'MEDIUM'")

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
8,8,CVE-2017-9140,47,MEDIUM,6.1,CVE-2017-9140,6.1 MEDIUM
24,24,CVE-2023-1175,17,MEDIUM,6.6,CVE-2023-1175,6.6 MEDIUM
28,28,CVE-2016-15030,15,MEDIUM,6.1,CVE-2016-15030,6.1 MEDIUM
30,30,CVE-2023-1616,14,MEDIUM,5.4,CVE-2023-1616,5.4 MEDIUM
53,53,CVE-2023-1289,10,MEDIUM,5.5,CVE-2023-1289,5.5 MEDIUM


In [136]:
df2[df2["baseSeverity"].notnull()]

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
8,8,CVE-2017-9140,47,MEDIUM,6.1,CVE-2017-9140,6.1 MEDIUM
10,10,CVE-2022-36413,31,CRITICAL,9.1,CVE-2022-36413,9.1 CRITICAL
14,14,CVE-2016-10190,26,CRITICAL,9.8,CVE-2016-10190,9.8 CRITICAL
16,16,CVE-2018-5093,26,HIGH,7.5,CVE-2018-5093,7.5 HIGH
19,19,CVE-2015-2291,22,HIGH,7.8,CVE-2015-2291,7.8 HIGH
21,21,CVE-2017-0145,19,HIGH,8.1,CVE-2017-0145,8.1 HIGH
23,23,CVE-2023-0494,17,HIGH,7.8,CVE-2023-0494,7.8 HIGH
24,24,CVE-2023-1175,17,MEDIUM,6.6,CVE-2023-1175,6.6 MEDIUM
25,25,CVE-2015-10097,16,CRITICAL,9.8,CVE-2015-10097,9.8 CRITICAL
27,27,CVE-2020-19786,15,HIGH,8.8,CVE-2020-19786,8.8 HIGH


In [137]:
df_gotScore = df2[df2["baseSeverity"].notnull()]

In [138]:
df_gotScore

,Unnamed: 0,CVE,num_of_author,baseSeverity,baseScore,CVE_checked,reason
8,8,CVE-2017-9140,47,MEDIUM,6.1,CVE-2017-9140,6.1 MEDIUM
10,10,CVE-2022-36413,31,CRITICAL,9.1,CVE-2022-36413,9.1 CRITICAL
14,14,CVE-2016-10190,26,CRITICAL,9.8,CVE-2016-10190,9.8 CRITICAL
16,16,CVE-2018-5093,26,HIGH,7.5,CVE-2018-5093,7.5 HIGH
19,19,CVE-2015-2291,22,HIGH,7.8,CVE-2015-2291,7.8 HIGH
21,21,CVE-2017-0145,19,HIGH,8.1,CVE-2017-0145,8.1 HIGH
23,23,CVE-2023-0494,17,HIGH,7.8,CVE-2023-0494,7.8 HIGH
24,24,CVE-2023-1175,17,MEDIUM,6.6,CVE-2023-1175,6.6 MEDIUM
25,25,CVE-2015-10097,16,CRITICAL,9.8,CVE-2015-10097,9.8 CRITICAL
27,27,CVE-2020-19786,15,HIGH,8.8,CVE-2020-19786,8.8 HIGH
